In [1]:
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
df = pd.read_json('../Data/combined.json')

In [3]:
df.shape

(25903, 19)

In [4]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first')
catagory = ohe.fit_transform(df.drop(['SalePrice',
 'SqFtTotLiving',
 'SqFtDeck',
 'SqFtOpenPorch',
 'Area',
 'SqFtLot',
 'WfntFootage'],axis=1))

In [5]:
column_names = ohe.get_feature_names([ 'PropertyType',
 'LakeWashington',
 'LakeSammamish',
 'SmallLakeRiverCreek',
 'WfntLocation',
 'WfntBank',
 'WfntPoorQuality',
 'WfntRestrictedAccess',
 'TidelandShoreland',
 'Bedrooms',
 'BathFullCount',
 'FinBasementGrade'])

In [6]:
df_cat = pd.DataFrame(index = df.index , data = catagory.todense(), columns=column_names)

In [7]:
df_cont = df[['SalePrice',
 'SqFtTotLiving',
 'SqFtDeck',
 'SqFtOpenPorch',
 'Area',
 'SqFtLot',
 'WfntFootage']]

In [8]:
final_df = pd.concat([df_cont, df_cat.reindex(df_cont.index)], axis=1)

In [9]:
final_df.shape

(25903, 68)

In [10]:
final_df.to_json('../Data/combined_onehot.json')